In [23]:
import msprime
import numpy as np
from itertools import combinations

In [25]:
def get_pi(haplotypes):
    ## If no seg sites in a pop then haplotypes will be 0 length
    if haplotypes.size == 0:
        return 0
    n = len(haplotypes[0])
    n_comparisons = float(n) * (n - 1) / 2

    pi = 0
    for hap in haplotypes:
        k = np.count_nonzero(hap)
        pi += float(k) * (n - k) / n_comparisons
    return(pi)


In [163]:
# Run `reps` number of simulations accumulating pi and pi* values then average over reps
def simulate(ss=10, Ne=1e4, length=5e2, reps=100):
    ts_pis = []
    my_pis = []
    uniq_pis = []
    resampled_pis = []
    nuniq = 0
    for i in range(reps):
        tree_sequence = msprime.simulate(sample_size=ss, Ne=Ne, length=length, mutation_rate=1e-7)
        ts_pis.append(tree_sequence.get_pairwise_diversity()/length)

        haps = list(tree_sequence.haplotypes())
        haps_t = np.transpose(np.array([list(map(int, list(x))) for x in haps]))
        my_pis.append(get_pi(haps_t)/length)

        haps = set(haps)
        haps_t = np.transpose(np.array([list(map(int, list(x))) for x in haps]))
        uniq_pis.append(get_pi(haps_t)/length)
        nuniq += len(haps)

        resampled_pis.append(resample(haps, length))

    print("  TS: {}/{}".format(np.mean(ts_pis), np.std(ts_pis)))
    print("  My: {}/{}".format(np.mean(my_pis), np.std(my_pis)))
    print("  Uniq (avg# {}): {}/{}".format(nuniq/float(reps),
                                        np.mean(uniq_pis),
                                        np.std(uniq_pis)))
    #print("  Resampled to <=5: {}".format(np.mean(resampled_pis)))

    return haps

# Ne 10000, 500bp loci
for ss in [5, 10, 25, 50]:
    print("Sample size {}".format(ss))
    _ = simulate(ss=ss, reps=100)

Sample size 5
  TS: 0.004576/0.00333193997545
  My: 0.004576/0.00333193997545
  Uniq (avg# 3.03): 0.006072/0.00410600297667
Sample size 10
  TS: 0.00381022222222/0.00239166333978
  My: 0.00381022222222/0.00239166333978
  Uniq (avg# 4.12): 0.00564847619048/0.00280617010328
Sample size 25
  TS: 0.0036908/0.00227661401208
  My: 0.0036908/0.00227661401208
  Uniq (avg# 5.7): 0.00617882539683/0.00336495878954
Sample size 50
  TS: 0.00402318367347/0.00246645366454
  My: 0.00402318367347/0.00246645366454
  Uniq (avg# 7.21): 0.0065221996892/0.00269547044362


In [164]:
# Ne 10000, for 5000bp loci
for ss in [5, 10, 25, 50]:
    print("Sample size {}".format(ss))
    simulate(ss=ss, length=5e3, reps=100)

Sample size 5
  TS: 0.003962/0.00224284373062
  My: 0.003962/0.00224284373062
  Uniq (avg# 4.54): 0.00421113333333/0.00238578122402
Sample size 10
  TS: 0.0041168/0.00218425160944
  My: 0.0041168/0.00218425160944
  Uniq (avg# 8.33): 0.00433464761905/0.00220678942886
Sample size 25
  TS: 0.00418788/0.00237521336844
  My: 0.00418788/0.00237521336844
  Uniq (avg# 16.34): 0.00449082116576/0.00240931534673
Sample size 50
  TS: 0.00428038204082/0.00255241770265
  My: 0.00428038204082/0.00255241770265
  Uniq (avg# 25.36): 0.00458455175386/0.00258529630936


In [165]:
# Ne 100000, for 500bp loci
for ss in [5, 10, 25, 50]:
    print("Sample size {}".format(ss))
    simulate(ss=ss, Ne=1e5, length=5e2, reps=100)

Sample size 5
  TS: 0.035912/0.0198464973232
  My: 0.035912/0.0198464973232
  Uniq (avg# 4.49): 0.037074/0.0192667333217
Sample size 10
  TS: 0.04368/0.0238300824512
  My: 0.04368/0.0238300824512
  Uniq (avg# 8.42): 0.0459373809524/0.0251966928151
Sample size 25
  TS: 0.0386714666667/0.0187111107594
  My: 0.0386714666667/0.0187111107594
  Uniq (avg# 16.47): 0.0410137145462/0.0200289716271
Sample size 50
  TS: 0.0412124081633/0.0213201970508
  My: 0.0412124081633/0.0213201970508
  Uniq (avg# 26.15): 0.0436178778215/0.0220485422932


In [166]:
# Ne 100000, for 5000bp loci
for ss in [5, 10, 25, 50]:
    print("Sample size {}".format(ss))
    simulate(ss=ss, Ne=1e5, length=5e3, reps=100)

Sample size 5
  TS: 0.0403556/0.0219305302407
  My: 0.0403556/0.0219305302407
  Uniq (avg# 4.96): 0.0405032/0.022140981379
Sample size 10
  TS: 0.0426194222222/0.0294884011793
  My: 0.0426194222222/0.0294884011793
  Uniq (avg# 9.84): 0.0428889968254/0.0297409762912
Sample size 25
  TS: 0.0365466/0.0159030120399
  My: 0.0365466/0.0159030120399
  Uniq (avg# 23.54): 0.0368156317749/0.0159856725722
Sample size 50
  TS: 0.0370141355102/0.0173566752718
  My: 0.0370141355102/0.0173566752718
  Uniq (avg# 45.1): 0.037198071374/0.0172897285051


## Trash below here
Experiment with multiple downsampling, but this is kind of dumb. Averaging
all possible combinations is identical to actually just using all the samples
in the first place.

In [153]:
def resample(haps, length=5e3, nresamples=5, verbose=False):
    totpi = 0
    minsamps = 5
    if len(haps) < minsamps:
        minsamps = len(haps) - 1
    if minsamps <=1:
        haps_t = np.transpose(np.array([list(map(int, list(x))) for x in haps]))
        return get_pi(haps_t)/length 
    for comb in list(combinations(haps, minsamps))[:nresamples]:
        haps_t = np.transpose(np.array([list(map(int, list(x))) for x in comb]))
        totpi += get_pi(haps_t)/length
    ncombs = len(list(combinations(haps, minsamps)))
    if verbose: print("ncombs: {}".format(ncombs))
    nresamples = min([nresamples, ncombs])
    return totpi/float(nresamples)
haps = simulate(ss=50, reps=1)
resample(haps, verbose=True)

  TS: 0.00428897959184
  My: 0.00428897959184
  Uniq (avg# 11.0): 0.0056
  Resampled to <=5: 0.00488
ncombs: 462


0.0004879999999999999